# PyCity Schools Analysis

- Your analysis here
  
---

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset for easier reading.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count = school_data_complete["school_name"].nunique()
school_count

15

In [3]:
# Calculate the total number of students - didn't use unique because some students have the same name
student_count = school_data_complete["student_name"].count()

student_count

39170

In [4]:
# Calculate the total budget - easier to calculate using the original school_data 
total_budget = school_data["budget"].sum()
total_budget

24649428

In [5]:
# Calculate the average (mean) math score - rounded for readability
average_math_score = school_data_complete["math_score"].mean().round(2)

average_math_score

78.99

In [6]:
# Calculate the average reading score - rounded for readability
average_reading_score = school_data_complete["reading_score"].mean().round(2)
average_reading_score

81.88

In [7]:
# calculate the percentage of students who passed math (math scores greather than or equal to 70)- rounded for readability
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage.round(2)

74.98

In [8]:
# Calculate the percentage of students who passed reading - rounded for readability 
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage.round(2)

85.81

In [9]:
#calculate the percentage of students that passed math and reading - rounded for readability 
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate.round(2)


65.17

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame - rounded for readability
district_summary = pd.DataFrame({"Total Schools":[school_count], "Total Students":[student_count],"Total Budget":[total_budget],
                                "Average Math Score":[average_math_score.round(2)], "Average Reading Score":[average_reading_score.round(2)], "% Passing Math":[passing_math_percentage.round(2)],
                                "% Passing Reading":[passing_reading_percentage.round(2)],"% Overall Passing Rate":[overall_passing_rate.round(2)]})

# Formatting
# district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
# district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.99,81.88,74.98,85.81,65.17


## School Summary

In [11]:
#select all of the school types
school_types = school_data_complete["type"]
print(f"There are {school_types.nunique()} types of schools")
school_types.unique()


There are 2 types of schools


array(['District', 'Charter'], dtype=object)

In [12]:
# Calculate the total student count per school

school_size = school_data_complete[["size", "school_name"]]
school_size = school_size.groupby(["school_name", "size"]).sum()
school_size = school_size.sort_values(by = "size", ascending=False)
school_size = school_size.reset_index()
school_size








,school_name,size
0,Bailey High School,4976
1,Johnson High School,4761
2,Hernandez High School,4635
3,Rodriguez High School,3999
4,Figueroa High School,2949
5,Huang High School,2917
6,Ford High School,2739
7,Wilson High School,2283
8,Cabrera High School,1858
9,Wright High School,1800


In [13]:
# Calculate the total school budget and per capita spending per school
per_school_capita = school_data_complete["budget"] / school_data_complete["size"]
#add new columns to existing dataframe
school_data_complete["per_school_capita"] = per_school_capita
#create new data frame with only the values I want (School name, budget per capita)
school_bpc = school_data_complete[["school_name", "per_school_capita"]]
#display new dataframe

school_bpc = school_bpc.groupby(["school_name","per_school_capita"]).count()
school_bpc = school_bpc.reset_index()
school_bpc =  school_bpc.sort_values(by = ["per_school_capita"], ascending=False)
school_bpc








,school_name,per_school_capita
7,Huang High School,655.0
5,Hernandez High School,652.0
8,Johnson High School,650.0
3,Ford High School,644.0
2,Figueroa High School,639.0
12,Thomas High School,638.0
10,Rodriguez High School,637.0
0,Bailey High School,628.0
4,Griffin High School,625.0
9,Pena High School,609.0


In [14]:
# Calculate the average test scores per school. Sorted highest to lowest
per_school_math_reading = school_data_complete[["school_name", "math_score", "reading_score"]]
per_school_math_reading = per_school_math_reading.groupby(["school_name"]).mean()
per_school_math_reading = per_school_math_reading.sort_values(by = ["math_score", "reading_score"], ascending=False)

#readnaming columns for readability - rename worked for math and reading, but I believe it's not working for
#school name because it is now an index. Tried to reindex but doesn't seem to work.

#have to set df equal to itself to successfully reindex. After that, the colmn name can be changed. Going back and reindexing previous dfs as well.

per_school_math_reading = per_school_math_reading.reset_index()
per_school_math_reading.rename(columns={"school_name": "School Name", "math_score": "Average Math Score", "reading_score": "Average Reading Score"}, inplace=True)
per_school_math_reading



,School Name,Average Math Score,Average Reading Score
0,Pena High School,83.839917,84.044699
1,Holden High School,83.803279,83.814988
2,Wright High School,83.682222,83.955000
3,Thomas High School,83.418349,83.848930
4,Shelton High School,83.359455,83.725724
5,Griffin High School,83.351499,83.816757
6,Wilson High School,83.274201,83.989488
7,Cabrera High School,83.061895,83.975780
8,Hernandez High School,77.289752,80.934412
9,Ford High School,77.102592,80.746258


In [15]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[["school_name", "math_score"]]

num_passing_math = students_passing_math.loc[students_passing_math["math_score"] >= 70]

num_passing_math = num_passing_math.groupby(["school_name"]).count()
num_passing_math = num_passing_math.reset_index()
num_passing_math
                                        

,school_name,math_score
0,Bailey High School,3318
1,Cabrera High School,1749
2,Figueroa High School,1946
3,Ford High School,1871
4,Griffin High School,1371
5,Hernandez High School,3094
6,Holden High School,395
7,Huang High School,1916
8,Johnson High School,3145
9,Pena High School,910


In [16]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[["school_name", "reading_score"]]
students_passing_reading = students_passing_reading.loc[students_passing_reading["reading_score"] >= 70]
students_passing_reading = students_passing_reading.groupby(["school_name"]).count()
students_passing_reading = students_passing_reading.reset_index()

students_passing_reading





,school_name,reading_score
0,Bailey High School,4077
1,Cabrera High School,1803
2,Figueroa High School,2381
3,Ford High School,2172
4,Griffin High School,1426
5,Hernandez High School,3748
6,Holden High School,411
7,Huang High School,2372
8,Johnson High School,3867
9,Pena High School,923


In [70]:
#calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
num_math_reading = pd.DataFrame(students_passing_math_and_reading, columns=["school_name"]).groupby(["school_name"])["school_name"].count()
num_math_reading_df = pd.DataFrame({"school_name":num_math_reading.index, "size_passing_both":num_math_reading.values})


# num_math_reading
num_math_reading




,school_name,size_passing_both
0,Bailey High School,2719
1,Cabrera High School,1697
2,Figueroa High School,1569
3,Ford High School,1487
4,Griffin High School,1330
5,Hernandez High School,2481
6,Holden High School,381
7,Huang High School,1561
8,Johnson High School,2549
9,Pena High School,871


In [73]:
# calculate the passing rates

passing_rate_both = pd.merge(pd.merge(num_passing_math, students_passing_reading, how="inner", on="school_name"), school_size, on = "school_name") 

passing_rate_both["Math Passing Rate"] = (passing_rate_both["math_score"] / passing_rate_both["size"] *100).round(2)
passing_rate_both["Reading Passing Rate"] = (passing_rate_both["reading_score"] / passing_rate_both["size"] *100).round(2)
passing_rate_both = pd.merge(passing_rate_both,num_math_reading_df, how="inner", on="school_name")
passing_rate_both["Math and Reading Passing Rate"] =  (passing_rate_both["size_passing_both"] / passing_rate_both["size"] *100).round(2)
passing_rate_both

,school_name,math_score,reading_score,size,Math Passing Rate,Reading Passing Rate,size_passing_both,Math and Reading Passing Rate
0,Bailey High School,3318,4077,4976,66.68,81.93,2719,54.64
1,Cabrera High School,1749,1803,1858,94.13,97.04,1697,91.33
2,Figueroa High School,1946,2381,2949,65.99,80.74,1569,53.20
3,Ford High School,1871,2172,2739,68.31,79.30,1487,54.29
4,Griffin High School,1371,1426,1468,93.39,97.14,1330,90.60
5,Hernandez High School,3094,3748,4635,66.75,80.86,2481,53.53
6,Holden High School,395,411,427,92.51,96.25,381,89.23
7,Huang High School,1916,2372,2917,65.68,81.32,1561,53.51
8,Johnson High School,3145,3867,4761,66.06,81.22,2549,53.54
9,Pena High School,910,923,962,94.59,95.95,871,90.54


In [19]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = 

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

SyntaxError: invalid syntax (2536131374.py, line 2)

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = 
top_schools.head(5)

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = 
bottom_schools.head(5)

## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = 
tenth_grader_math_scores = 
eleventh_grader_math_scores = 
twelfth_grader_math_scores = 

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = 

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = 
tenth_grader_reading_scores = 
eleventh_grader_reading_scores = 
twelfth_grader_reading_scores = 

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = 

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary